In [4]:
# pip install -q -U tensorflow-addons

Note: you may need to restart the kernel to use updated packages.


In [1]:
import io
import os
import time
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.image import imread

from scipy.spatial.distance import cdist
import cv2 as cv
import tensorflow as tf
import tensorflow_addons as tfa
from tripletloss_preprocessing import PreProcessing
from tripletloss_model import TripletLoss
from tripletloss_prediction import show_image

In [2]:
model_path = 'trained_model/model_triplet/'
model = TripletLoss()

In [3]:
# Input and Output Tensor
tf.compat.v1.disable_eager_execution()
img_placeholder = tf.compat.v1.placeholder(tf.float32, [None, 28, 28, 3], name='img')
net = model.conv_net(img_placeholder, reuse=False) # from TripletLoss class

C:\Users\Aisyah\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\legacy_tf_layers\convolutional.py:414: UserWarning: `tf.layers.conv2d` is deprecated and will be removed in a future version. Please Use `tf.keras.layers.Conv2D` instead.
  warnings.warn('`tf.layers.conv2d` is deprecated and '
C:\Users\Aisyah\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1719: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
C:\Users\Aisyah\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\keras\legacy_tf_layers\core.py:329: UserWarning: `tf.layers.flatten` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Flatten` instead.
  warnings.warn('`tf.layers.flatten` is deprecated and '


In [ ]:
train_images, valid_images, train_label, valid_label = PreProcessing('retail_corpus/').preprocessing(0.9)

Loading Retail Corpus Dataset...


In [1]:
# helper function to plot image
def show_image(idxs, data):
    if type(idxs) != np.ndarray:
        idxs = np.array([idxs]) # 2d array
    fig = plt.figure()
    gs = gridspec.GridSpec(1,len(idxs))
    for i in range(len(idxs)): # iterate through image indexes
        ax = fig.add_subplot(gs[0,i])
        ax.imshow(data[idxs[i],:,:,:])
        ax.axis('off')
    plt.show()

In [ ]:
# Generate random index from valid_images corpus and display the image
idx = np.random.randint(0, len(valid_images))
img = valid_images[idx]

print("********** QUERY IMAGE **********")
show_image(idx, valid_images)

### Finding k-nearest neighbours (k most similar images) using cosine similarity
Note: 
- there is a library that will facilitate a faster and more efficient similarity search called `faiss` but it is an 'Unofficial prebuilt binary for Linux and MacOS' only.
- The CPU-only faiss-cpu conda package is currently available on Linux, OSX, and Windows. The faiss-gpu, containing both CPU and GPU indices, is available on Linux systems, for various versions of CUDA. https://github.com/facebookresearch/faiss/blob/master/INSTALL.md

In [ ]:
# compute vector representation for each training image and normalise 
def generate_norm_vectors():
    saver = tf.compat.v1.train.Saver()
    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.global_variables_initializer())
        ckpt = tf.train.get_checkpoint_state(model_path) # model_path defined earlier as model_triplet
        saver.restore(sess, model_path + "model.ckpt")
        train_vectors = sess.run(net, feed_dict={img_placeholder:train_images})      
    normalized_train_vectors = train_vectors/np.linalg.norm(train_vectors,axis=1).reshape(-1,1) # reshape array into 2d array regardless of original shape
    return normalized_train_vectors

# Find k nearest neighbours using cos similarity
def find_k_nn(normalized_train_vectors,vec,k):
    dist_arr = np.matmul(normalized_train_vectors, vec.T)
    return np.argsort(-dist_arr.flatten())[:k] # in descending order of similarity

In [ ]:
norm_training_vectors = generate_norm_vectors()

In [ ]:
# compute vector representation of valid image 
saver = tf.compat.v1.train.Saver()
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    ckpt = tf.train.get_checkpoint_state(model_path)
    saver.restore(sess, model_path + "model.ckpt")
    search_vector = sess.run(net, feed_dict={img_placeholder:[img]}) # defined before knn section   
normalized_search_vec = search_vector/np.linalg.norm(search_vector)

In [ ]:
s_time = time.time()
k = 10 # k most similar images
candidate_index = find_k_nn(norm_training_vectors, normalized_search_vec, k)
print('Total time to find NNs: {:0.2f} ms'.format((time.time()-s_time)*1000)) # current time - time started
fig = plt.figure(figsize=(10,0.8))
idxs = [idx]
gs = gridspec.GridSpec(1, len(idxs))

# plot test image
for i in range(len(idxs)):
    ax = fig.add_subplot(gs[0, i]) # all in a row
    ax.imshow(valid_images[idxs[i], :, :, :])
    ax.axis('off')
plt.show()

# plot similar images
show_image(candidate_index, train_images)
print("Index of Similar Images:", candidate_index)

In [ ]:
# helper function to plot multiple images
def show_top_k_images(indx_list,valid_image_indexes, train_data, valid_data):
    fig = plt.figure(figsize=(20,40))
    gs = gridspec.GridSpec(len(indx_list),len(indx_list[0])+2)
    for i in range(len(indx_list)):
        ax = fig.add_subplot(gs[i,0]) # each row different image query
        ax.imshow(valid_data[valid_image_indexes[i],:,:,:])
        ax.axis('off')
        for j in range(len(indx_list[0])): # for each image query, show its similar images
            ax = fig.add_subplot(gs[i,j+2])
            ax.imshow(train_data[indx_list[i][j],:,:,:])
            ax.axis('off')
    plt.savefig('figures/similar_images.jpg') 
    plt.show()

In [ ]:
K = 10
N = 20
indx_list = []
valid_image_indexes = []
_valid_images = []
for i in range(N):
    idx = i
    valid_image_indexes.append(idx)
    _valid_images.append(valid_images[idx])
    # run the test image through the network to get the test features
saver = tf.compat.v1.train.Saver()
with tf.compat.v1.Session() as sess:
    sess.run(tf.compat.v1.global_variables_initializer())
    ckpt = tf.train.get_checkpoint_state(model_path)
    saver.restore(sess, model_path + "model.ckpt")
    search_vectors = sess.run(net, feed_dict={img_placeholder:_valid_images})
    
normalized_search_vecs = search_vectors/np.linalg.norm(search_vectors,axis=1).reshape(-1,1)
for i in range(len(normalized_search_vecs)):
    candidate_index = find_k_nn(norm_training_vectors, normalized_search_vecs[i], K)
    indx_list.append(candidate_index)

In [ ]:
print('** Query Image **         *************************** Top {} Similar Images  ***************************'.format(K))
show_top_k_images(indx_list,valid_image_indexes, train_images, valid_images)

In [24]:
matplotlib.image.imread(os.path.join('retail_corpus/Office Reception Sofas/','22209_contemporary_group-1a_2.gif'))

array([[[140, 150, 156, 255],
        [156, 166, 164, 255],
        [148, 158, 164, 255],
        ...,
        [ 52,  50,  52, 255],
        [ 52,  50,  52, 255],
        [ 68,  66,  68, 255]],

       [[140, 150, 156, 255],
        [156, 158, 164, 255],
        [148, 158, 164, 255],
        ...,
        [ 52,  50,  52, 255],
        [ 52,  50,  52, 255],
        [ 68,  66,  68, 255]],

       [[140, 150, 156, 255],
        [148, 166, 164, 255],
        [148, 158, 164, 255],
        ...,
        [ 52,  50,  52, 255],
        [ 52,  50,  52, 255],
        [ 76,  66,  68, 255]],

       ...,

       [[ 44,  34,  28, 255],
        [ 36,  22,  12, 255],
        [ 44,  34,  28, 255],
        ...,
        [ 44,  34,  28, 255],
        [ 44,  34,  28, 255],
        [ 44,  34,  28, 255]],

       [[ 44,  42,  36, 255],
        [ 36,  22,  12, 255],
        [ 36,  22,  12, 255],
        ...,
        [ 44,  34,  28, 255],
        [ 44,  34,  28, 255],
        [ 44,  34,  28, 255]],

       [[ 44

In [23]:
dict(zip(['sofa', 'table','lamp'], range(3)))

{'sofa': 0, 'table': 1, 'lamp': 2}

In [29]:
idxs = [1,1,9]
if type(idxs) != np.ndarray:
    idxs = np.array([idxs])
    print(idxs)

[[1 1 9]]
